# **Solución al problema práctico Semana 1**

## **Los hábitos de consumo y los tarjetahabientes**

La agrupación es una técnica que permite identificar patrones y tendencias en los datos con base en la identificación de grupos o segmentos de objetos con características comunes. Para el caso de una empresa que ofrece el servicio de tarjetas de crédito, la segmentación de clientes le permitiría, por ejemplo, ofrecer productos adaptados a las características de los grupos identificados, dirigir campañas de marketing a segmentos específicos de clientes y mejorar la experiencia de estos con un servicio de atención personalizado.

### **A. Problema práctico.**

Una empresa de tarjetas de crédito desea segmentar a sus clientes en función de su comportamiento de compra en centros comerciales con el fin de identificar los diferentes tipos de tarjetahabientes y establecer estrategias de marketing personalizadas para cada uno. Para realizar este estudio se cuenta con datos sobre saldos, límites de crédito y hábitos de compra, entre otros, de una muestra de clientes suministrada por la empresa. 

La empresa nos ha contratado para llevar adelante este estudio, el cual realizaremos en estas dos primeras semanas del curso. Las tareas para realizar en cada una de ellas son:

**Semana 1.** Exploración, limpieza y preparación del conjunto de datos, considerando que se utilizará un algoritmo de agrupación basado en distancias.

**Semana 2.** Aplicación del algoritmo de agrupación y caracterización de cada segmento, con el fin de definir patrones de comportamiento.

Conjunto de datos. Los datos ("Customer_data") han sido tomados de este 
repositorio  y han sido modificados para propósitos de este ejercicio. Es importante que revises el diccionario (lo encuentras en el repositorio) como primer paso para comprender estos datos.

### **B. Actividad - Semana 1.**
 Construir un notebook que realice la exploración (perfilamiento), limpieza y preparación de los datos. Para la fase de exploración puedes utilizar la librería pandas-profiling (aunque no es un requisito): 

https://github.com/ydataai/ydata-profiling. 



### **1. Importar librerías necesarios**
Iniciaremos importando las librerias `Pandas` y `scikit-learn`, además de la librería `numpy` para procesamiento matemático y `joblib` para persistencia de objetos. En particular, importaremos algunas clases dentro de `scikit-learn` que nos serán de utilidad para realizar el procesamiento de datos:

* [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html): clase para realizar la imputación de datos faltantes.
* [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html): clase para realizar la normalización de valores numéricos.
* [`MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html): clase para realizar la normalización de valores numéricos usando los valores límite de los datos.
* [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html): clase para realizar la codificación One-Hot de las variables categóricas.
* [`ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html): clase para combinar pasos de preprocesamiento.

In [12]:
import pandas as pd
import numpy as np
import joblib
from ydata_profiling import ProfileReport
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

### **2. Carga de datos**

`Pandas` permite la lectura de fuentes de datos almacenadas en archivos con varios formatos. Entre los formatos más comunes para el almacenamiento de conjuntos de datos se encuentran los archivos de valores separados por comas (CSV) y JavaScript Object Notation (JSON). 
En este caso los datos están en un archivo CSV por que se leerán utilizando la función de `Pandas` `read_csv()`.

Los datos están en la carpeta `data` y el archivo se llama `Customer_Data.csv`. Se cargan los datos en un DataFrame de `Pandas` llamado `data_raw` de la siguiente manera:


In [6]:
data_raw = pd.read_csv('./data/Customer_Data.csv')

Para ver una muestra de los datos, se utiliza la función `head()` que, por defecto, muestra las primeras 5 filas del DataFrame:

In [7]:
data_raw.head()

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,5323.148883,0.250693,26344.072201,38237.442525,3727.113162,15708.239684,0.496536,0.464442,0.821611,0.194502,100,77,16819.480037,21337.027458,39452.958121,0.817907,7
1,C10002,12726.638115,0.791307,37958.519019,5690.742440,18733.810964,38284.354433,0.699457,0.250327,0.654863,1.083902,78,156,15617.570575,8000.183624,63013.748477,0.343119,9
2,C10003,4305.572068,0.176531,28392.953338,36009.470088,2873.383232,14294.185035,0.419764,0.523662,0.899912,0.207049,72,81,15515.586213,27111.360493,NaN,0.829074,6
3,C10004,4740.988511,0.178076,27399.003842,38246.863491,3402.853375,6936.812518,0.439666,0.606597,0.783129,0.228299,78,83,12926.587974,23919.113404,38444.219979,0.883984,7
4,C10005,13820.920640,0.826914,42214.021633,7341.007821,19273.070991,40091.347849,0.821412,0.283579,0.501361,1.106350,88,182,14404.705067,6994.688474,62041.617340,0.383186,10


Se puede conocer el tamaño del DataFrame utilizando el atributo `shape`, que devuelve una tupla con el número de filas y columnas del DataFrame:

In [9]:
data_raw.shape

(8950, 18)

### **3. Descripción de los datos**
Para identificar los tipo de datos de las columnas del DataFrame, se utiliza el atributo `dtypes`, que devuelve una serie con los nombres de las columnas y sus respectivos tipos de datos:

In [10]:
data_raw.dtypes

CUST_ID                              object
BALANCE                             float64
BALANCE_FREQUENCY                   float64
PURCHASES                           float64
ONEOFF_PURCHASES                    float64
INSTALLMENTS_PURCHASES              float64
CASH_ADVANCE                        float64
PURCHASES_FREQUENCY                 float64
ONEOFF_PURCHASES_FREQUENCY          float64
PURCHASES_INSTALLMENTS_FREQUENCY    float64
CASH_ADVANCE_FREQUENCY              float64
CASH_ADVANCE_TRX                      int64
PURCHASES_TRX                         int64
CREDIT_LIMIT                        float64
PAYMENTS                            float64
MINIMUM_PAYMENTS                    float64
PRC_FULL_PAYMENT                    float64
TENURE                                int64
dtype: object

La gran mayoría de nuestras variables son del tipo `float64`, es decir, números flotantes representados números decimaes de 64 bits. Adicionalmente tenemos tres variable del tipo `int64`, que corresponde a un número entero de 64 bits. Finalmente tenemos una variable del tipo `object`, indicando que se tratan de objetos más complejos como cadenas de caracteres.

Específicamente para las variables numéricas, tanto enteras como decimales, podemos obtener una descripción de los datos mediante la función `describe()`, que retorna algunas medidas estadísticas como la media, la desviación estándar, el mínimo y el máximo de los valores para cada una.

In [11]:
data_raw.describe()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
count,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8949.000000,8950.000000,8637.000000,8950.000000,8950.000000
mean,9382.387900,0.459205,30525.873439,22913.665720,11407.851805,22573.474202,0.559810,0.478032,0.623916,0.734946,76.169162,164.981453,14696.640361,19824.347218,44222.977795,0.545414,8.370615
std,5118.113559,0.282998,11041.410943,13573.919086,7087.102996,12594.295895,0.214055,0.221873,0.208207,0.422563,22.260935,71.322003,5537.074673,11069.268177,15468.314241,0.267443,1.756035
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,1006.064965,0.000000,6.000000
25%,4275.036859,0.188490,27106.651600,6698.252490,3736.720194,10944.431111,0.407273,0.250856,0.518231,0.230716,65.000000,97.000000,13487.727313,9329.603116,31572.127683,0.336732,7.000000
50%,12007.718812,0.306650,30934.582274,31011.033488,10832.141085,19465.792260,0.505305,0.533521,0.607608,0.947651,82.000000,176.000000,14942.545347,21367.621276,48780.271754,0.432690,8.000000
75%,13740.565876,0.756061,39825.160798,34495.416423,19127.083036,36150.722564,0.777084,0.649120,0.814929,1.029172,92.000000,216.000000,16269.727124,25219.127375,57761.175985,0.815178,10.000000
max,19043.138560,1.000000,49039.570000,40761.250000,22500.000000,47137.211760,1.000000,1.000000,1.000000,1.500000,123.000000,358.000000,30000.000000,50721.483360,76406.207520,1.000000,12.000000


In [14]:
pd.Series(data_raw['CASH_ADVANCE_TRX']).value_counts()

CASH_ADVANCE_TRX
85     260
84     250
87     243
90     234
88     234
      ... 
118      1
6        1
7        1
121      1
0        1
Name: count, Length: 118, dtype: int64

In [15]:
pd.Series(data_raw['PURCHASES_TRX']).value_counts()

PURCHASES_TRX
209    72
192    67
196    66
206    66
214    65
       ..
27      1
22      1
337     1
20      1
324     1
Name: count, Length: 331, dtype: int64

In [16]:
pd.Series(data_raw['TENURE']).value_counts()

TENURE
10    3336
6     2091
7     1414
8     1021
11     575
9      512
12       1
Name: count, dtype: int64

### **4. Limpieza y preparación de datos**
Para construir los modelos de aprendizaje no supervisado es necesario realizar algunas modificaciones según el algoritmo. Por lo tanto, es recomendable utilizar la función `copy()` para almacenar una copia del conjunto de datos y modificarlo sin alterar el original:

In [17]:
data = data_raw.copy()

#### **Eliminación de variables poco relevantes**
Este conjunto de datos tiene una variables poco relevantepara realizar un proceso de agrupación: `CUST_ID`. Haciendo uso de la función `drop()`, se eliminará esta variable del DataFrame. Se utilizará una lista para definir las variables a eliminar, incluyendo el parámetro `axis=1` para indicar que se realizará la operación sobre las columnas:

In [18]:
data = data_raw.drop(columns=['CUST_ID'], axis=1)
data

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,5323.148883,0.250693,26344.072201,38237.442525,3727.113162,15708.239684,0.496536,0.464442,0.821611,0.194502,100,77,16819.480037,21337.027458,39452.958121,0.817907,7
1,12726.638115,0.791307,37958.519019,5690.742440,18733.810964,38284.354433,0.699457,0.250327,0.654863,1.083902,78,156,15617.570575,8000.183624,63013.748477,0.343119,9
2,4305.572068,0.176531,28392.953338,36009.470088,2873.383232,14294.185035,0.419764,0.523662,0.899912,0.207049,72,81,15515.586213,27111.360493,NaN,0.829074,6
3,4740.988511,0.178076,27399.003842,38246.863491,3402.853375,6936.812518,0.439666,0.606597,0.783129,0.228299,78,83,12926.587974,23919.113404,38444.219979,0.883984,7
4,13820.920640,0.826914,42214.021633,7341.007821,19273.070991,40091.347849,0.821412,0.283579,0.501361,1.106350,88,182,14404.705067,6994.688474,62041.617340,0.383186,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8945,6095.221156,0.234346,27094.007822,35917.204378,3463.821376,11711.578889,0.469832,0.539477,0.828321,0.191030,75,101,17623.810282,23954.396952,34841.829885,0.866856,7
8946,12682.575822,0.686025,38433.618847,5293.541956,18924.799798,34444.867317,0.710331,0.276426,0.546059,0.935152,95,170,12433.814929,7375.625322,66726.372993,0.306600,10
8947,16464.311409,0.189136,4296.196384,32554.759639,11231.500433,15214.671563,0.424665,0.746010,0.690380,1.063788,51,151,3307.587117,22947.759217,54761.962206,0.129242,8
8948,15531.883777,0.117118,12219.325647,31795.971700,12390.432127,13856.393208,0.450670,0.805140,0.608737,1.058548,61,178,6148.232910,21369.503519,50368.309060,0.087448,8
